---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## CheckPoint GeoPandas

## Datasets

En la práctica de la clase de hoy usaremos un **conjunto de propiedades** que tienen a la venta la inmobiliaria Properati: https://www.properati.com.ar/. Lo interesante es que a los datos clásicos de la propiedad (valor, superficie, barrio, tipo de propiedad), le agrega la posición geoespacial mediante su latitud y longitud.

Por otra parte, vamos a considerar a las **estaciones de subte**, donde tambien figuran el nombre y la línea a la que pertenece, y además, obviamente, sus datos geoposicionales.

## Ejercicio

Comenzamos leyendo los dos datasets en un *dataframe*, y lo transformamos en un *GeoDataFrame*. Es el tipo de datos que requiere GeoPandas para realizar operaciones con datos geoespaciales.

Luego vamos a *calcular la distancia* de cada propiedad al obelisco de Buenos Aires, y se registrará en una nueva columna. Haremos un cálculo para ver si existe alguna relacion entre esta distancia y el precio de la propiedad.

Finalmente *trazaremos una línea geométrica* con todas las estaciones de subte de una línea, y la graficaremos sobre la ciudad.

Alla vamos!

---

Importamos las bibliotecas que vamos a necesitar:

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import shapely # genera las figuras geometricas
import descartes # relaciona shapely con matplotlib
import pyproj # proyecciones. Transformar coordenadas

---

### Parte 1 - Archivo de propiedades

Vamos a leer los datos del archivo /M1/CLASE_07/Data/properati_caba.csv en un `DataFrame` de pandas con el método `read_csv`. Lo llamaremos df_prop. 

*Nota:* los datos vienen separados por tabs. Se debe usar el parámetro <code>sep='\t'</code>

Ayuda:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [ ]:
data_location = "../Data/properati_caba.csv"

df_prop = pd.read_csv(data_location, sep='\t')

#### Tomamos una muestra de 5 elementos

Las columnas del dataset son:

- property_type. Tipo de propiedad. Categórico.
- place_name. Barrio de Buenos Aires. Categórico.
- country_name. Pais. Fijo 'Argentina'.
- state_name. Provincia. Fijo 'Capital Federal'.
- lat. Latitud de la posición geoespacial del inmueble.
- lon. Longitud de la posición geoespacial del inmueble.
- price. Valor de la propiedad.
- currency. Moneda usada para el valor de la propiedad.
- surface_total_in_m2. Area del inmueble.

In [ ]:
df_prop.sample(5)

#### Transformar el DataFrame en un GeoDataFrame

Para generar un GeoDataFrame, necesitamos agregar a las columnas del DataFrame una nueva columna que contenga una forma geométrica. Es este caso, un *punto*, el cual se genera a partir de la *latitud* y la *longitud*.

Es recomendable que la nueva columna se llame *geometry*.

El método `gpd.points_from_xy` define el tipo *punto*, ingresando primero la **longitud** y luego la **latitud**.

Ayuda:
https://geopandas.org/reference/geopandas.points_from_xy.html

In [ ]:
geometria = gpd.points_from_xy(df_prop.lon, df_prop.lat)

In [ ]:
geometria[0:2]

In [ ]:
geo_prop = gpd.GeoDataFrame(df_prop ,geometry=geometria)

In [ ]:
type(geo_prop)

In [ ]:
geo_prop.loc[:4, ['geometry', 'lat', 'lon','property_type','place_name']]

---

### Parte 2 - Archivo de estaciones de subte

Vamos a leer los datos del archivo /M1/CLASE_07/Data/estaciones-de-subte.csv en un `DataFrame` de pandas con el método `read_csv`. Lo llamaremos df_subte.

*Nota:* los datos vienen separados por comas. Se debe usar el parámetro <code>sep=','</code>

In [ ]:
data_location = "../Data/estaciones-de-subte.csv"

df_subte = pd.read_csv(data_location, sep=',')

#### Consultamos los primeros 10 registros

Las columnas del dataset son:

- long. Longitud de la posición geoespacial de la estación de subte.
- lat. Latitud de la posición geoespacial de la estación de subte.
- id. Número secuencial.
- estacion. Nombre de la estación de subte.
- linea. Línea de subte a la que pertenece la estación. Categórico.

In [ ]:
df_subte.head(10)

#### Transformar el DataFrame en un GeoDataFrame

Para generar un GeoDataFrame, necesitamos agregar a las columnas del DataFrame una nueva columna que contenga una forma geométrica. Es este caso, un *punto*, el cual se genera a partir de la *latitud* y la *longitud*.

Es recomendable que la nueva columna se llame *geometry*.

El método `gpd.points_from_xy` define el tipo *punto*, ingresando primero la **longitud** y luego la **latitud**.

In [ ]:
geometria = gpd.points_from_xy(df_subte.long, df_subte.lat)

In [ ]:
geo_subte = gpd.GeoDataFrame(df_subte ,geometry=geometria)

In [ ]:
type(geo_subte)

---

### Parte 3 - Cálculo de la distancia

Vamos a calcular la **distancia de cada propiedad al obelisco de Buenos Aires**, y la registramos en una nueva columna.

Primero necesitamos representar el lugar geográfico del Obelisco de Buenos Aires. Lo hacemos mediante la forma geométrica *Punto*, y las cooordenadas del lugar. 

In [ ]:
from shapely.geometry import Point

punto_obelisco_p = Point(-58.381555,-34.605425)

In [ ]:
type(punto_obelisco_p)

Pero el método que calcula la distancia entre dos puntos, <code>geopy.distance.geodesic</code>, necesita representar el lugar geográfico del Obelisco de Buenos Aires mediante una *tupla*.

In [ ]:
punto_obelisco = (-58.381555,-34.605425)

In [ ]:
type(punto_obelisco)

Ahora podemos agregar la nueva columna con la distancia entre el punto del obelisco y el punto de cada propiedad (en metros). La llamaremos *distancia_obelisco*. 

Se debe generar **una iteración** sobre el indice de geo_prop:

Ayuda:
<code>
for i in geo_prop.index:
    geo_prop.loc[i, 'distancia_obelisco'] = calculo distancia entre obelisco y cada propiedad.</code>
    
El cálculo se realiza con el método <code>geopy.distance.geodesic</code>, el cual se explica en la notebook 3_geopandas_operaciones.

Ademas con la sentencia <code>geo_prop.loc[i,'geometry'].x</code> obtenemos la coordenada longitud de la distancia.

Debe terminar con <code>.meters</code> para indicar la distancia en metros

Ver https://geopy.readthedocs.io/en/stable/#module-geopy.distance

In [ ]:
geo_prop.index

In [ ]:
import geopy.distance

for i in geo_prop.index:
    geo_prop.loc[i, 'distancia_obelisco'] = geopy.distance.geodesic(punto_obelisco,
                  (geo_prop.loc[i,'geometry'].x
                  ,geo_prop.loc[i,'geometry'].y)).meters

In [ ]:
geo_prop.loc[:4, ['distancia_obelisco','geometry', 'lat', 'lon','property_type','place_name']]

Es interesante encontrar **valores extremos** en la distancia calculada (llamados *outliers*). Nos pueden distorsionar los calculos estadísticos. 

In [ ]:
geo_prop.sort_values(by="distancia_obelisco", ascending=False).head(5)

In [ ]:
geo_prop["distancia_obelisco"].max()

El primer registro tiene una distancia incorrecta, que se genera a partir del dato erróneo de la longitud. Mejor eliminarlo.

In [ ]:
indice = geo_prop[geo_prop.distancia_obelisco > 19000].index

indice

In [ ]:
geo_prop.drop(indice, inplace=True)

In [ ]:
geo_prop.sort_values(by="distancia_obelisco", ascending=False).head(5)

Y ahora podemos hacer un histograma sobre las distancias.

In [ ]:
plt.title('Distancia al Obelisco')
plt.hist(geo_prop["distancia_obelisco"], bins=20, alpha=1, edgecolor = 'black',  linewidth=1)
plt.grid(True)
plt.show()
plt.clf()

---

### Parte 4 - Línea que una a las estaciones de subte

Vamos a hacer una línea geométrica con las estaciones de subte de la línea H y graficarla sobre la ciudad.

<div>
    <div class = "mapa">
        <img src='img/M1_Clase_07_1_desafio.jpg' alt="Elementos geométricos" width=80% height=90%>
    </div>
</div>

In [ ]:
geo_subte.head(4)

Primero seleccionemos solo las estaciones de la linea H, y guardamos la selección en la variable geo_subte_h.

In [ ]:
geo_subte_h = geo_subte.loc[geo_subte.linea =='H', :]

In [ ]:
geo_subte_h

Como muestra el dibujo, la línea corre de Norte a Sur (o Sur a Norte). Como las estaciones deben estar alineadas para que salga bien la línea, podemos usar la columna lat para ordenarlas; lat es la latitud de cada estación.

Ayuda: considerar sort_values con by
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html

In [ ]:
geo_subte_h = geo_subte_h.sort_values(by='lat')

In [ ]:
geo_subte_h

Para simplificar la codificación, solo nos quedaremos con los datos geográficos de cada estación.

Generamos una *GeoSeries* con la columna geometry de geo_subte_h. Y la llamamos geo_subte_h_geometry.

In [ ]:
geo_subte_h_geometry = geo_subte_h["geometry"]

In [ ]:
type(geo_subte_h_geometry)

Ahora creamos las líneas que unen a las estaciones de subte. 

Cada línea se forma con **dos puntos**. Por lo tanto, tenemos que tomar el primer y segundo punto de la Geoserie, y aplicarles el método <code>LineString</code>. Luego seguimos con el segundo y el tercero, y así siguiendo.

*Primero creamos una lista donde guardaremos cada linea.* La llamamos linea_h. Debe tener una longitud igual a la cantidad de estaciones menos 1, es decir longitud 11.

In [ ]:
longitud_linea = len(geo_subte_h_geometry)-1
linea_h = list(range(longitud_linea))

In [ ]:
len(linea_h)

Ahora podemos crear las líneas y guardarlas en la lista.

Se debe generar **una iteración** para recorrer la GeoSerie *geo_subte_h_geometry*.

Ayuda:
<code>
for i in range(longitud_linea): 
    linea_h[i] = creo la linea entre el elemento i y el elemento i+1
</code> 
    
El cálculo se realiza con el método <code>LineString</code> de la libreria shapely, el cual se explica en la notebook 3_geopandas_operaciones.

In [ ]:
from shapely.geometry import LineString

for i in range(longitud_linea): 
    linea_h[i] = LineString([geo_subte_h_geometry.iloc[i], geo_subte_h_geometry.iloc[i+1]])

In [ ]:
linea_h

---

Ahora vamos a graficar las líneas sobre un mapa con los barrios de Buenos Aires.

Vamos a leer los datos del archivo /M1/CLASE_07/Data/barrios.csv en un `DataFrame` de pandas con el método `read_csv`. Lo llamaremos df_barrios.

In [ ]:
df_barrios = pd.read_csv("../Data/barrios.csv", encoding='latin1')

In [ ]:
df_barrios.head(3)

In [ ]:
type(df_barrios)

---

Y luego generamos un GeoDataFrame de los barrios a partir del Dataframe. En este caso, la columna WKT ya contiene una forma geométrica **polígono** que representa el contorno de cada barrio. Pero en un formato WKT, que no permite generar un GeoDataFrame. Por lo tanto, lo tenemos que convertir a un formato *geometry*.

In [ ]:
import shapely.wkt

df_barrios["WKT"] = df_barrios["WKT"].apply(shapely.wkt.loads) 
geo_barrios = gpd.GeoDataFrame(df_barrios, geometry='WKT')

In [ ]:
geo_barrios.dtypes

Por otra parte, necesitamos para graficar las líneas generar una GeoSerie donde cada elemento se compone de dos puntos y la línea entre ellos.

In [ ]:
linea_h_geo = list(range(longitud_linea))

for i in range(longitud_linea): 
    linea_h_geo[i] = gpd.GeoSeries([geo_subte_h_geometry.iloc[i], geo_subte_h_geometry.iloc[i+1], linea_h[i]])

In [ ]:
linea_h_geo[:2]

Con el método plot generamos el gráfico:

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
geo_barrios.plot(ax=ax, color='white', edgecolor='black')

for i in range(len(geo_subte_h_geometry)-2): 
    linea_h_geo[i].plot(ax=ax, color='red')

plt.show();